In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
import xgboost as xgb
import time
import random as rnd

standarscaler = StandardScaler()

import warnings
warnings.filterwarnings("ignore")

In [2]:
TrainData = pd.read_csv('count_train.csv')
X = TrainData.drop('yC', axis = 1)
Y = TrainData['yC']

Xs = standarscaler.fit_transform(X)
Xs = pd.DataFrame(Xs, index=X.index, columns=X.columns.values)

In [3]:
TestData = pd.read_csv('count_test.csv')
Xt = TestData.drop('yC', axis = 1)
Yt = TestData['yC']

Xts = standarscaler.transform(Xt)
Xts = pd.DataFrame(Xts, index=Xt.index, columns=Xt.columns.values)

In [4]:
variables = [0 for x in range(10000000)]
for i in range(10000000):
    variables[i] = rnd.sample(X.columns.values.tolist(), rnd.sample([x+1 for x in range(X.shape[1])],1)[0])

In [ ]:
RMSE_List = [0 for x in range(10000000)]
RMSE_List_round = [0 for x in range(10000000)]
RMSE_List_floor = [0 for x in range(10000000)]
RMSE_List_ceil = [0 for x in range(10000000)]
start_time = time.time()
for i in range(len(variables)):
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
    xg_reg.fit(Xs[variables[i]],Y)
    preds = xg_reg.predict(Xts[variables[i]])
    RMSE_List[i] = (np.sqrt(mean_squared_error(Yt, preds)))
    RMSE_List_round[i] = np.sqrt(mean_squared_error(Yt, np.round(preds)))
    RMSE_List_floor[i] = np.sqrt(mean_squared_error(Yt, np.floor(preds)))
    RMSE_List_ceil[i] = np.sqrt(mean_squared_error(Yt, np.ceil(preds)))
elapsed_time = time.time() - start_time
print(elapsed_time)

In [26]:
vari = np.array(variables)
RMSE = np.array(RMSE_List)
Round = np.array(RMSE_List_round)
Floor = np.array(RMSE_List_floor)
Ceil = np.array(RMSE_List_ceil)

In [27]:
vari1 = vari[RMSE !=0]
RMSE1 = RMSE[RMSE !=0]
Round1 = Round[RMSE !=0]
Floor1 = Floor[RMSE !=0]
Ceil1 = Ceil[RMSE !=0]

In [28]:
print(min(RMSE1))
print(min(Round1))
print(min(Floor1))
print(min(Ceil1))

1.0425212693687411
1.0165300454651272
1.140175425099138
1.1105554165971787


In [29]:
chosen = np.array(vari1)[Round1 == min(Round1)]
chosen = chosen.tolist()[0]
chosen

['x24',
 'x18',
 'x8',
 'x2',
 'x22',
 'x16',
 'x4',
 'x29',
 'x14',
 'x30',
 'x13',
 'x25',
 'x27',
 'x7',
 'x26',
 'x3']

In [30]:
xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror')
xg_reg2.fit(Xs[chosen],Y)
preds2 = xg_reg2.predict(Xts[chosen])
np.sqrt(mean_squared_error(Yt, np.round(preds2)))

1.0165300454651272

In [31]:
np.round(preds2)

array([1., 2., 1., 1., 0., 1., 1., 1., 1., 1., 2., 0., 1., 1., 1., 2., 1.,
       2., 2., 2., 2., 1., 1., 1., 1., 1., 4., 1., 1., 1.], dtype=float32)

In [32]:
Yt.values

array([0, 2, 2, 2, 0, 1, 0, 0, 1, 0, 4, 0, 1, 1, 0, 1, 2, 2, 3, 2, 3, 0,
       0, 0, 0, 2, 7, 0, 0, 1], dtype=int64)